In [1]:
import pickle
import torch
from transformers import BertModel
from tqdm import tqdm 
from octis.preprocessing.preprocessing import Preprocessing
from Code.TNTM.TNTM_SentenceTransformer import TNTM_SentenceTransformer
import numpy as np

/media/soroush/Partition 3/temp-topic-modelling/.bertopicenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


current device: cuda
current device: cuda


In [2]:
torch.manual_seed(41)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'current device: {device}')

current device: cuda


In [3]:
with open("Data/DataResults_BERT/cleaned_embedding_df_20ng_BERT.pickle", "rb") as f: 
  embedding_df = pickle.load(f)

In [4]:
embedding_df.sort_values(by = "word", inplace = True)

In [5]:
embedding_df

,embedding,is_valid
word,,
a,"[tensor(-0.4876), tensor(0.2411), tensor(0.158...",True
aa,"[tensor(0.0616), tensor(1.8438), tensor(0.5808...",True
aaron,"[tensor(-0.2090), tensor(0.5433), tensor(0.163...",True
ab,"[tensor(-0.2713), tensor(0.9461), tensor(0.288...",True
ability,"[tensor(-0.3597), tensor(0.2136), tensor(0.293...",True
...,...,...
zeus,"[tensor(-0.5143), tensor(1.4048), tensor(0.162...",True
zip,"[tensor(0.6577), tensor(0.0648), tensor(0.9260...",True
zone,"[tensor(-0.2998), tensor(-0.1618), tensor(0.38...",True


In [6]:
embedding_ten_lis = []

for i in range(len(embedding_df)):
    embedding_ten_lis.append(embedding_df["embedding"].iloc[i])

In [7]:
embedding_ten = torch.stack(embedding_ten_lis)

In [8]:
embedding_ten.shape

torch.Size([3349, 768])

In [9]:
with open("Data/Preprocessed_Data/octis_dataset_20ng.pickle", "rb") as f: 
  dataset_raw_20ng = pickle.load(f)

vocab = dataset_raw_20ng.get_vocabulary()
corpus = dataset_raw_20ng.get_corpus()

In [10]:
len(vocab)

3349

In [11]:
len(corpus)

18846

In [12]:
from sentence_transformers import SentenceTransformer

# Load the model
embeddings_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

In [13]:

with open("Data/Auxillary_Data/twng_textData.txt", "r") as file:
    data20ng_text = file.readlines()

In [14]:
for line in data20ng_text:
    print(line)
    break

From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu> Subject: Pens fans reactions Organization: Post Office, Carnegie Mellon, Pittsburgh, PA Lines: 12 NNTP-Posting-Host: po4.andrew.cmu.edu    I am sure some bashers of Pens fans are pretty confused about the lack of any kind of posts about the recent Pens massacre of the Devils. Actually, I am  bit puzzled too and a bit relieved. However, I am going to put an end to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they are killing those Devils worse than I thought. Jagr just showed you why he is much better than his regular season stats. He is also a lot fo fun to watch in the playoffs. Bowman should let JAgr have a lot of fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final regular season game.          PENS RULE!!!  



In [15]:
import torch
sentence_embedding = []
for line in tqdm(data20ng_text):
    line_embedded =embeddings_model.encode(line.lower())
    sentence_embedding.append(line_embedded)

    
sentence_embedding = torch.tensor(sentence_embedding)
sentence_embedding.shape

100%|██████████| 18846/18846 [01:30<00:00, 208.87it/s]
/tmp/ipykernel_11568/4154606747.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  sentence_embedding = torch.tensor(sentence_embedding)


torch.Size([18846, 384])

In [16]:
#sentence_embedding = torch.randn(18846, 384)

In [17]:

tntm = TNTM_SentenceTransformer(
    n_topics = 10,
    save_path = f"Data/Auxillary_Data/{20}_topics",
    n_dims = 11,
    n_hidden_units = 200,
    n_encoder_layers = 2,
    enc_lr = 1e-3,
    dec_lr = 1e-3,
    n_epochs = 20,
    #batch_size = 128,
    batch_size = 256,
    dropout_rate_encoder = 0.3,
    prior_variance =  0.995, 
    prior_mean = None,
    n_topwords = 200,
    device = device, 
    validation_set_size = 0.2, 
    early_stopping = True,
    n_epochs_early_stopping = 10,
)

In [18]:
# from TNTM_SentenceTransformer import TNTM_SentenceTransformer
# tntm = TNTM_SentenceTransformer(
#       n_topics  = 10, 
#       save_path = f"example/{20}_topics", 
#       enc_lr    = 1e-3,
#       dec_lr    = 1e-3
#       )

In [19]:
result = tntm.fit(
              corpus              = corpus,
              vocab               = vocab, 
              word_embeddings     = embedding_ten,
              document_embeddings = sentence_embedding)

/media/soroush/Partition 3/temp-topic-modelling/.bertopicenv/lib/python3.9/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12040. The TBB threading layer is disabled.
  warnings.warn(problem)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/media/soroush/Partition 3/temp-topic-modelling/TNTM/Code/TNTM/TNTM_SentenceTransformer.py:155: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mus_init_ten = torch.tensor(mus_init).to(s

Epoch nr 0: mean_train_loss = -3769.306884765625, mean_train_nl = -3783.849853515625, mean_train_kld = 14.542951583862305, elapsed time: 1.7186377048492432
Epoch nr 0: median_train_loss = -3751.2451171875, median_train_nl = -3754.071044921875, median_train_kld = 16.507986068725586, elapsed time: 1.7186377048492432
Epoch nr 0: mean_val_loss = -3958.85791015625, mean_val_nl = -3975.9365234375, mean_val_kld = 17.078641891479492
Epoch nr 0: median_val_loss = -3877.677490234375, median_val_nl = -3894.702880859375, median_val_kld = 17.08100128173828
gradient norm: mean: 5297.024604291946, median: 3567.2001572752642, max: 19862.27348527945


Epoch nr 1: mean_train_loss = -4008.54443359375, mean_train_nl = -4026.195068359375, mean_train_kld = 17.65085220336914, elapsed time: 1.775292158126831
Epoch nr 1: median_train_loss = -3955.8876953125, median_train_nl = -3974.18115234375, median_train_kld = 17.687191009521484, elapsed time: 1.775292158126831
Epoch nr 1: mean_val_loss = -4126.90771484375,

### Test the Model

In [20]:
weights = result[1]
# normalize weights for each corresponding unique word
normalize_weights = weights/weights.sum(axis=1, keepdims=True)

In [21]:
import numpy as np

# Parameters
num_clusters = result[1].shape[0]  # Number of clusters
num_words = result[1].shape[1]  # Number of unique words

# Extract the most relevant word and its score for each cluster
most_relevant_words = []

for cluster_idx in range(num_clusters):
    # Get the weight scores for all words in the current cluster
    word_scores = normalize_weights[cluster_idx]
    
    most_relevant_word_idx = word_scores.argmax()
    
    # Retrieve the actual word using result[0] (the vocabulary)
    most_relevant_word = result[0][cluster_idx][most_relevant_word_idx]
    
    most_relevant_score = word_scores[most_relevant_word_idx]
    
    most_relevant_words.append((most_relevant_word, most_relevant_score))

for cluster_idx, (word, score) in enumerate(most_relevant_words):
    print(f"Cluster {cluster_idx + 1}: {word} (Score: {score:.4f})")


Cluster 1: hi (Score: 0.0309)
Cluster 2: focus (Score: 0.0241)
Cluster 3: more (Score: 0.0199)
Cluster 4: ground (Score: 0.0331)
Cluster 5: the (Score: 0.1173)
Cluster 6: gw (Score: 0.0453)
Cluster 7: nation (Score: 0.0333)
Cluster 8: address (Score: 0.0272)
Cluster 9: ted (Score: 0.1018)
Cluster 10: go (Score: 0.0867)


# BERTopic 
### using BERTopic to assing one of the above topic to a document  

In [ ]:
from bertopic import BERTopic